In this programme we use data on churn of customers to help us predict the likelihood that a given customer will churn. We have the following steps

- Read the data 
- Initialise parameters for w and b
- Define cost function
- Define number of layers and nodes in the neural network
- Define the cost function
- Optimise using gradient descent to minimise the cost function
- Output the parameters that optimise
- Run against dev set to see accurancy
- Ask for data for a customer and then calculate probablility of churn

In [26]:
import tensorflow as tf
import numpy as np
import pandas as pd

#Import the data
data = pd.read_csv('churn_modelling.csv')

#Check number of columns
print(data.columns)
print()
print("There are", len(data.columns),"columns")

#Use the final column as y = the output
output = data[['Exited']]
print("shape of output is", np.shape(output))

#Use the input columns as input features x (note we ignore customer ID and surname)
#FOr the initial version we will also remove anything that is not a integer data (i.e. geography, gender)
input_features = data[['CreditScore',  'Age', 'Tenure','Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']]
print("shape of input_fetures is", np.shape(input_features))

#Create different sets of data for train, dev and test
y_train = output.loc[0:9000,:]
y_dev = output.loc[9001:9500,:]
y_test = output.loc[9501:10000,:]

x_train = input_features.loc[0:9000,:]
x_dev = input_features.loc[9001:9500,:]
x_test = input_features.loc[9501:10000,:]

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

There are 14 columns
shape of output is (10000, 1)
shape of input_fetures is (10000, 8)


Define key parameters that impact the accuracy of the algorithm

In [27]:
learning_rate = 0.000001
epochs = 10
batch_size = 100
num_nodes = 5
num_layers = 2

Create placeholders for the input features and the output

In [28]:
#Create placeholders
x = tf.placeholder(tf.float32, [None, 8], name = 'x')
y = tf.placeholder(tf.float32, [None, 1], name = 'y')

Initialise w and b for a single layer neural network

In [29]:
#Initialise weights as random between 0 and 1
w1 = tf.Variable(tf.random_uniform(shape = [8,num_nodes], minval = 0, maxval = 0.1, dtype = tf.float32, name = 'w1' ))
w2 = tf.Variable(tf.random_uniform(shape = [num_nodes,1], minval = 0, maxval = 0.1, dtype = tf.float32, name = 'w2' ))

#Initialise biases as zero
b1 = tf.Variable(tf.zeros(shape = [num_nodes], dtype = tf.float32, name = 'b1'))
b2 = tf.Variable(tf.zeros(shape = [1], dtype = tf.float32, name = 'b2'))



Version will use gradient descent and linear regression with the help of the below URL
https://blog.manash.me/implementation-of-gradient-descent-in-tensorflow-using-tf-gradients-c111d783c78bFirst
Predict the answer y using relu and calculate the cost

In [33]:
#Forward propagation from input to hidden layer
a1 = tf.nn.relu(tf.matmul(x,w1) + b1)

#Forward propagation from hidden layer to the output
preds = tf.nn.relu(tf.matmul(a1,w2) + b2)
                   
#Calculate the cost
cost = tf.losses.mean_squared_error(y, preds)

#Define optimiser
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost)
#optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)

#Define a corect prediction
correct_pred = tf.equal(y,preds)

#Define accuracy
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

Initialise global variables and run the Tensorflow sessions

In [34]:
#Initialise global variables
model = tf.global_variables_initializer()

#Start the session
with tf.Session() as sess:
    sess.run(model)
    _, c = sess.run([optimizer, cost], feed_dict={x: x_train, y: y_train})
    print(c)
    
    print("Accuracy of the model on the dev set is", sess.run([accuracy], feed_dict={x:x_dev, y:y_dev}))
    print("Accuracy of the model on the test set is", sess.run([accuracy], feed_dict={x:x_test, y:y_test}))
    

6.10767e+06
Accuracy of the model on the dev set is [0.81599998]
Accuracy of the model on the test set is [0.82765532]


Ask for an input of the different input features and then give a probability that they will exit

In [32]:
input_features.columns[1]

'Age'

In [21]:
x_new_data = {}
for i in range (len(input_features.columns)):
    inputs = input(input_features.columns[i])
    x_new_data[input_features.columns[i]] = int(inputs)



CreditScore345
Age23
Tenure23
Balance234
NumOfProducts1
HasCrCard1
IsActiveMember1
EstimatedSalary2345


In [22]:
x_new_data = pd.DataFrame(x_new_data, index = [0])
print(np.shape(x_new_data))
print(x_new_data)




(1, 8)
   Age  Balance  CreditScore  EstimatedSalary  HasCrCard  IsActiveMember  \
0   23      234          345             2345          1               1   

   NumOfProducts  Tenure  
0              1      23  


In [23]:
with tf.Session() as sess:
    sess.run(model)
    print(sess.run([preds], feed_dict = {x: x_new_data}))


[array([[ 59.17082596]], dtype=float32)]
